<a href="https://colab.research.google.com/github/rachit2005/Large-Language-Model/blob/main/gpt_%3E_from_andrej_karpathy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [74]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-06-11 01:41:19--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.4’

input.txt.4         100%[===================>]   1.06M  --.-KB/s    in 0.04s   

2025-06-11 01:41:19 (26.6 MB/s) - ‘input.txt.4’ saved [1115394/1115394]



In [75]:
import requests

url= "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
resp = requests.get(url)

text = resp.text
print(len(text))

1115394


In [76]:
chars = sorted(list(set(text)))

vocab_size = len(chars)

print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


Tokenize

In [77]:
# building the character level tokenizer
stoi = {ch:i for i ,ch in enumerate(chars)}
itos = {value:key for key , value in stoi.items()}

encoder = lambda s: [stoi[c] for c in s]
decoder = lambda l: ''.join(itos[i] for i in l)

print(stoi)
print(itos)

print(encoder("hii there"))
print(decoder(encoder("hii there")))

{'\n': 0, ' ': 1, '!': 2, '$': 3, '&': 4, "'": 5, ',': 6, '-': 7, '.': 8, '3': 9, ':': 10, ';': 11, '?': 12, 'A': 13, 'B': 14, 'C': 15, 'D': 16, 'E': 17, 'F': 18, 'G': 19, 'H': 20, 'I': 21, 'J': 22, 'K': 23, 'L': 24, 'M': 25, 'N': 26, 'O': 27, 'P': 28, 'Q': 29, 'R': 30, 'S': 31, 'T': 32, 'U': 33, 'V': 34, 'W': 35, 'X': 36, 'Y': 37, 'Z': 38, 'a': 39, 'b': 40, 'c': 41, 'd': 42, 'e': 43, 'f': 44, 'g': 45, 'h': 46, 'i': 47, 'j': 48, 'k': 49, 'l': 50, 'm': 51, 'n': 52, 'o': 53, 'p': 54, 'q': 55, 'r': 56, 's': 57, 't': 58, 'u': 59, 'v': 60, 'w': 61, 'x': 62, 'y': 63, 'z': 64}
{0: '\n', 1: ' ', 2: '!', 3: '$', 4: '&', 5: "'", 6: ',', 7: '-', 8: '.', 9: '3', 10: ':', 11: ';', 12: '?', 13: 'A', 14: 'B', 15: 'C', 16: 'D', 17: 'E', 18: 'F', 19: 'G', 20: 'H', 21: 'I', 22: 'J', 23: 'K', 24: 'L', 25: 'M', 26: 'N', 27: 'O', 28: 'P', 29: 'Q', 30: 'R', 31: 'S', 32: 'T', 33: 'U', 34: 'V', 35: 'W', 36: 'X', 37: 'Y', 38: 'Z', 39: 'a', 40: 'b', 41: 'c', 42: 'd', 43: 'e', 44: 'f', 45: 'g', 46: 'h', 47: 'i',

# tiktoken --> use by openai's models

In [78]:
import tiktoken

enc = tiktoken.get_encoding("gpt2")
enc.n_vocab

print(enc.encode("hii there"))
print(enc.decode(enc.encode("hii there")))

[71, 4178, 612]
hii there


In [79]:
# using sentencepiece tokenizer

! pip install sentencepiece

import sentencepiece as spm

spm.SentencePieceTrainer.train(input="input.txt", model_prefix="m", vocab_size=5000)


import sentencepiece as spm

# Load model
sp = spm.SentencePieceProcessor()
sp.load('m.model')

# Encode text into subword tokens
tokens = sp.encode("This is an example", out_type=str)
print("Tokens:", tokens)

# Encode as IDs
ids = sp.encode("This is an example", out_type=int)
print("IDs:", ids)

# Decode back to string
decoded = sp.decode(ids)
print("Decoded:", decoded)

Tokens: ['▁This', '▁is', '▁an', '▁example']
IDs: [191, 25, 135, 3016]
Decoded: This is an example


# DO NOT CLOSE THIS

In [80]:
import torch

# encoding the whole data into tensor

data = torch.tensor(encoder(text) , dtype=torch.long)
print(data.shape)

torch.Size([1115394])


In [81]:
# splitint the data into test and train

n = 0.9*len(data)

train_data = data[:int(n)]
val_data = data[int(n):]

print(train_data.shape)
print(val_data.shape)

torch.Size([1003854])
torch.Size([111540])


In [82]:
from torch.utils.data import DataLoader , Dataset

BLOCK_SIZE = 8
# train_data[:BLOCK_SIZE]

# how we going to provide the data in the transformer
x = train_data[:BLOCK_SIZE]
y = train_data[1:BLOCK_SIZE+1]

pad = [0]

for t in range(BLOCK_SIZE):
  context = torch.cat((x[:t+1] , torch.tensor(pad*(BLOCK_SIZE - t))))
  target = y[t]
  print(f"when input is {context} , then target is {target}")


when input is tensor([18,  0,  0,  0,  0,  0,  0,  0,  0]) , then target is 47
when input is tensor([18, 47,  0,  0,  0,  0,  0,  0,  0]) , then target is 56
when input is tensor([18, 47, 56,  0,  0,  0,  0,  0,  0]) , then target is 57
when input is tensor([18, 47, 56, 57,  0,  0,  0,  0,  0]) , then target is 58
when input is tensor([18, 47, 56, 57, 58,  0,  0,  0,  0]) , then target is 1
when input is tensor([18, 47, 56, 57, 58,  1,  0,  0,  0]) , then target is 15
when input is tensor([18, 47, 56, 57, 58,  1, 15,  0,  0]) , then target is 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47,  0]) , then target is 58


In [83]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [84]:
from ast import Str

BATCH_SIZE = 4

def get_batch(split:Str , batch_size=BATCH_SIZE):
  data = train_data if split == "train" else val_data

  ix = torch.randint(0 , len(data)-BLOCK_SIZE, (batch_size,))

  x = torch.stack([data[i:i+BLOCK_SIZE] for i in ix])
  y = torch.stack([data[i+1:i+BLOCK_SIZE + 1] for i in ix])

  return x , y

xb, yb = get_batch('train' , BATCH_SIZE)
xb = xb.to(device)
yb = yb.to(device)
print(xb.shape)
print(xb)
print(yb.shape)
print(yb)

for b in range(BATCH_SIZE):
  for t in range(BLOCK_SIZE):
    context = torch.cat((xb[b, :t+1] , torch.tensor(pad*(BLOCK_SIZE - t+1)).to(device)))
    target = yb[b , t]

    print(f"when input is {context} , then target is {target}")
  break # i will see only one batch

torch.Size([4, 8])
tensor([[ 5, 42,  1, 48, 53, 47, 52, 58],
        [ 0,  0, 15, 50, 53, 61, 52, 10],
        [63, 39, 50,  1, 57, 47, 56,  6],
        [25, 26, 21, 13, 10,  0, 27,  6]], device='cuda:0')
torch.Size([4, 8])
tensor([[42,  1, 48, 53, 47, 52, 58,  1],
        [ 0, 15, 50, 53, 61, 52, 10,  0],
        [39, 50,  1, 57, 47, 56,  6,  1],
        [26, 21, 13, 10,  0, 27,  6,  1]], device='cuda:0')
when input is tensor([5, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0') , then target is 42
when input is tensor([ 5, 42,  0,  0,  0,  0,  0,  0,  0,  0], device='cuda:0') , then target is 1
when input is tensor([ 5, 42,  1,  0,  0,  0,  0,  0,  0,  0], device='cuda:0') , then target is 48
when input is tensor([ 5, 42,  1, 48,  0,  0,  0,  0,  0,  0], device='cuda:0') , then target is 53
when input is tensor([ 5, 42,  1, 48, 53,  0,  0,  0,  0,  0], device='cuda:0') , then target is 47
when input is tensor([ 5, 42,  1, 48, 53, 47,  0,  0,  0,  0], device='cuda:0') , then target is 52
w

Self - Attention

d_k --> no of heads

# attention = softmax((q @ transpose of k) / sqrt(d_k)) @ v --> acc to paper

In [85]:
emb_dim = 32
BLOCK_SIZE = 8
BATCH_SIZE = 4

In [86]:
import torch
from torch import nn
import math

class Head(nn.Module):
  "single head of self-attention"

  def __init__(self , head_size):
    super().__init__()
    self.head_size = head_size

    self.key = nn.Linear(emb_dim , head_size , bias=False)
    self.query = nn.Linear(emb_dim , head_size , bias=False)
    self.value = nn.Linear(emb_dim , head_size , bias=False)
    self.register_buffer('tril' , torch.tril(torch.ones(BLOCK_SIZE , BLOCK_SIZE)))

  def forward(self , x):
      B,T,C = x.shape
      k = self.key(x) # --> [B,T,C]
      q = self.query(x) # --> [B,T,C]
      v = self.value(x) # --> [B,T,C]

      attention_ = q@k.transpose(1,2)/ math.sqrt(C) # --> [B,T,T]
      attention_ = attention_.masked_fill(self.tril == 0 , float('-inf'))
      attention_ = F.softmax(attention_ , dim=-1)

      return attention_ @ v # --> [B,T,T] @ [B,T,C] --> [B,T,C]

Multi-head-attention

In [87]:
class MultiHeadAttention(nn.Module):
  def __init__(self , num_heads , head_size):
    super().__init__()
    self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
    self.proj = nn.Linear(emb_dim , emb_dim)
    self.dropout = nn.Dropout(0.2)

  def forward(self , x):
    out = torch.cat([h(x) for h in self.heads] , dim=-1)
    out = self.dropout(self.proj(out))
    return out

Feed Forward

In [88]:
class FeedForward(nn.Module):
  def __init__(self):
    super().__init__()
    self.feed_forward = nn.Sequential(
        nn.Linear(emb_dim , 2048),
        nn.ReLU(),
        nn.Linear(2048 , emb_dim)
    )
  def forward(self , x):
    return self.feed_forward(x)

residual connection


In [89]:
class Block(nn.Module):
  def __init__(self , emb_dim , n_head):
    super().__init__()
    self.head_size = emb_dim // n_head
    self.multi_head_attention = MultiHeadAttention(n_head , self.head_size)
    self.feed_forward = FeedForward()
    self.layer_norm = nn.LayerNorm(emb_dim)

  def forward(self , x):
    x = self.layer_norm(x +  self.multi_head_attention(x))
    x = self.layer_norm(x +  self.feed_forward(x))
    return x


LayerNorm --> in this we normalise the rows instead of columns like in BatchNorm

In [90]:
import torch
from torch import nn
import torch.nn.functional as F


class BilingualLanguageModel(nn.Module):
  def __init__(self, vocab_size) -> None:
    super().__init__()

    self.embedding = nn.Embedding(vocab_size , emb_dim)
    self.position_embedding_table = nn.Embedding(BLOCK_SIZE , emb_dim)
    self.blocks = nn.Sequential(
        Block(emb_dim , 4),
        Block(emb_dim , 4),
        Block(emb_dim , 4),
        nn.LayerNorm(emb_dim)
    )

    # self.self_attention_head = MultiHeadAttention(4 , emb_dim//4) # --> 4 heads of 8 dimensional self-attention --> it will concat to give us the 32 dimensional vector --> [4,8,32]
    # self.feed_forward = FeedForward()
    self.lm_head = nn.Linear(emb_dim , vocab_size)

  def forward(self , idx , target):
    # x --> [batch , block] --> [batch , block , embedding_dim] (B,T,emb_dim) --> [batch , block , vocab_size]

    '''tok_emb are the token embeddings for the input indices.
      pos_emb is generated for positions 0 to T-1.'''

    B,T = idx.shape
    tok_emb = self.embedding(idx) # --> [B,T,C] --> [4,8,32]
    pos_emb = self.position_embedding_table(torch.arange(T , device=device)) # --> [T,C] --> [8,32]
    x = tok_emb + pos_emb # --> [4 , 8 , 32]
    # print(x.shape)
    # x = self.self_attention_head(x) # --> [4 , 8 , 32]
    # x = self.feed_forward(x) # --> [4,8,32]
    x = self.blocks(x)
    logits = self.lm_head(x) # --> [B,T,vocab_size] --> [4 , 8,65]

    if target is None:
      loss = None
    else:
      B,T,C = logits.shape
      logits = logits.view(B*T , C) # --> [32 , 65]
      target = target.view(-1)      # B*T
      loss = F.cross_entropy(logits , target)

    return logits , loss

  def generate(self ,idx , max_new_tokens):
    # idx is the (B,T) array of indices in the current context
    for _ in range(max_new_tokens):
      idx_condensed = idx[:, -BLOCK_SIZE:] # crop context to BLOCK_SIZE

      logits , loss = self(idx_condensed , None)

      logits = logits[: , -1 , :] # [batch ,vocab_size=65] --> selecting the last element of the batch in the time step
      probs = F.softmax(logits , dim=-1)
      idx_next = torch.multinomial(probs , num_samples=1) # --> (B,1)

      # append sampled index to the running sequences
      idx = torch.cat((idx , idx_next) , dim=1) # --> (B,T+1)...to (B,T+max_new_tokens)

    return idx

model = BilingualLanguageModel(vocab_size).to(device)
logits , loss = model(xb , yb)

print(logits.shape , loss)


print(model.generate(xb , 5))

# decoding the generated output
print(decoder(model.generate(xb , 100)[3].tolist()))

torch.Size([32, 65]) tensor(4.3028, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor([[ 5, 42,  1, 48, 53, 47, 52, 58, 13, 56, 51, 35,  3],
        [ 0,  0, 15, 50, 53, 61, 52, 10,  1, 60, 52, 38, 42],
        [63, 39, 50,  1, 57, 47, 56,  6, 10, 42,  9,  9, 23],
        [25, 26, 21, 13, 10,  0, 27,  6, 41, 32, 20, 28, 26]], device='cuda:0')
MNIA:
O,bsysjHHCFAia Nerl?PEb-v3IAOvVTDq'TldsD  oeqVTvxK; k!
?cAKMmwH3WJTw&:PNVj3eilP
nwB:xb ud&oMWvqeiWWqKG


In [91]:
eval_iters = 200
max_iters = 30000
eval_interval = 300

def estimate_loss():
  out = {}
  model.eval()
  with torch.no_grad():
    for split in ['train' , 'val']:
      losses = torch.zeros(eval_iters)
      for k in range(eval_iters):
        X , Y = get_batch(split)
        X = X.to(device)
        Y = Y.to(device)
        logits , loss = model(X , Y)
        losses[k] = loss.item()
      out[split] = losses.mean()
    model.train()
    return out

In [92]:
optimizer = torch.optim.Adam(model.parameters() , lr=1e-3)
model = model.to(device)

for iter in range(max_iters):
  if iter%eval_interval == 0:
    losses = estimate_loss()
    print(f"step: {iter} || train_loss: {losses['train']:.2f} || val_loss: {losses['val']:.2f}")
  xb , yb = get_batch("train" , 32)
  xb = xb.to(device)
  yb = yb.to(device)

  logits , loss = model(xb , yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

print(loss.item())

step: 0 || train_loss: 4.39 || val_loss: 4.38
step: 300 || train_loss: 2.47 || val_loss: 2.49
step: 600 || train_loss: 2.34 || val_loss: 2.36
step: 900 || train_loss: 2.30 || val_loss: 2.27
step: 1200 || train_loss: 2.16 || val_loss: 2.20
step: 1500 || train_loss: 2.12 || val_loss: 2.16
step: 1800 || train_loss: 2.10 || val_loss: 2.12
step: 2100 || train_loss: 2.07 || val_loss: 2.12
step: 2400 || train_loss: 2.12 || val_loss: 2.08
step: 2700 || train_loss: 2.03 || val_loss: 2.07
step: 3000 || train_loss: 2.00 || val_loss: 2.08
step: 3300 || train_loss: 2.00 || val_loss: 2.05
step: 3600 || train_loss: 1.96 || val_loss: 2.07
step: 3900 || train_loss: 1.95 || val_loss: 2.06
step: 4200 || train_loss: 1.93 || val_loss: 2.05
step: 4500 || train_loss: 1.94 || val_loss: 2.03
step: 4800 || train_loss: 1.94 || val_loss: 2.04
step: 5100 || train_loss: 1.89 || val_loss: 2.00
step: 5400 || train_loss: 1.90 || val_loss: 2.02
step: 5700 || train_loss: 1.87 || val_loss: 1.99
step: 6000 || train_loss: 

In [95]:
print(decoder(model.generate(xb , 1000)[3].tolist()))

arry, good
Mellor: you not slear
I amonoughters,
Whost come, and we warrely me tire, plage thands; what that make far or laters me with u'er
Most thee wars to bloward
Marcum'tings with your furror lad stand to curse:
These to prain himset uprict must or an queen.

Second Selms of your eye,
Only their gly the chories that thou old delam, how chold'dine land-beman:
What not reprowt discourt. O friend Yox: with thank take for I immost begoa it, o'e through!
Lettingers: crowar he did neven king, here earerquars the goad.

GLOUCESTER:
A knows come kines the invow? O let to joughtones on
prevish, in thee mean?

AUFIDIUS:
I, yea:
But is to-not man? I in followed very such advintedy to ceret om them lasts murder possin;
Four earward for princenion.
A trouble.

CAMILLO:
I dear about come crown oftlukes.
Then? ind,
First Citizes bring.
No, true have to a migs his wort answer.

BRUTUS:

Serving, how sine ting sound me agged.
'Sest no makes and bury could very flict.

CLAUS:
I shall abselvings of 

In [94]:
import math

x = torch.randn(4,8,32)

head_size = 16 # --> N
query = nn.Linear(emb_dim , head_size , bias=False)
key = nn.Linear(emb_dim , head_size ,bias=False)
value = nn.Linear(emb_dim , head_size ,bias=False)

q = query(x) # --> [32 , 8,head_size]
k = key(x) # --> [32,8,head_size]
v = value(x) # --> [32,8,head_size]

wei = q@k.transpose(1,2)/ math.sqrt(head_size) # --> ([32 , 8 , head_size] @ [32 , head_size , 8])@ [32 , 8 , head_size] --> [32,8,8] @ [32,8 , head_size] --> [32 , 8 , head_size]
# print(wei[0])
# now we will mask so that the model cannot see the future tokens

tril = torch.tril(torch.ones(x.shape[1] , x.shape[1]))
wei = wei.masked_fill(tril == 0 , float('-inf'))
wei = F.softmax(wei , dim = -1)

print(wei[0])

xbow = wei @ v # --> [32,8,head_size] @ [32,8,65] --> [32,8,65]
print(xbow.shape) # --> [32 , 8 , 65]
# print(xbow)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.6519, 0.3481, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2026, 0.2550, 0.5423, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2110, 0.2471, 0.2248, 0.3171, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1807, 0.2425, 0.1828, 0.2542, 0.1398, 0.0000, 0.0000, 0.0000],
        [0.1555, 0.1196, 0.0892, 0.2331, 0.1782, 0.2245, 0.0000, 0.0000],
        [0.1802, 0.1169, 0.1308, 0.1486, 0.1730, 0.1471, 0.1035, 0.0000],
        [0.1475, 0.1262, 0.0941, 0.1314, 0.0836, 0.1671, 0.1138, 0.1362]],
       grad_fn=<SelectBackward0>)
torch.Size([4, 8, 16])
